# Tarea 7: Muestreo estratificado

Solución de los ejercicios de la sección `Intervalos de confianza y remuestreo` del libro del curso [Fundamentos Estadísticos](https://tereom.github.io/fundamentos-2022/intervalos-de-confianza-y-remuestreo.html)

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyreadr

from scipy.stats import gamma, norm, binom, probplot

In [2]:
path_universal = "C:\\Users\\Miguel\\Documents\\Github\\itam-mcd"
os.listdir(path_universal)

['.git',
 '.gitignore',
 'aprendizaje-automatico',
 'estadistica-computacional',
 'fundamentos-estadisticos',
 'mineria-analisis-datos',
 'optimizacion',
 'README.md']

In [3]:
path_asignatura = "fundamentos-estadisticos"
path_asignatura = os.path.join(path_universal, path_asignatura)
os.listdir(path_asignatura)

['datos', 'libros', 'README.md']

## Conteo rápido

En México, las elecciones tienen lugar un domingo, los resultados oficiales del proceso se presentan a la población una semana después. A fin de evitar proclamaciones de victoria injustificadas durante ese periodo, el INE organiza un conteo rápido. El conteo rápido es un procedimiento para estimar, a partir de una muestra aleatoria de casillas, el porcentaje de votos a favor de los candidatos en la elección. 

En este ejercicio deberás crear intervalos de confianza para la proporción de votos que recibió cada candidato en las elecciones de 2006. La inferencia se hará a partir de una muestra de las casillas similar a la que se utilizó para el conteo rápido de 2006.

El diseño utilizado es **muestreo estratificado simple**, lo que quiere decir que:
1) Se particionan las casillas de la población en estratos (cada casilla pertenece a exactamente un estrato), y 
2) Dentro de cada estrato se usa *muestreo aleatorio* para seleccionar las casillas que estarán en la muestra. 

En este ejercicio (similar al conteo rápido de 2006):
* Se seleccionó una muestra de $7,200$ casillas
* La muestra se repartió a lo largo de 300 estratos. 
* La tabla `strata_sample_2006` contiene en la columna $N$ el número total de casillas en el estrato, y en $n$ el número de casillas que se seleccionaron en la muestra, para cada estrato.

In [9]:
#Se obtiene dataset original
file_path = "datos\\strata_sample_2006.rda"
strata_sample_2006 = pyreadr.read_r(os.path.join(path_asignatura, file_path))
strata_sample_2006 = strata_sample_2006['strata_sample_2006']
strata_sample_2006.head()

,stratum,n,N
0,1.0,20,369
1,2.0,23,420
2,3.0,24,440
3,4.0,31,570
4,5.0,29,528


In [10]:
strata_sample_2006.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   stratum  300 non-null    float64
 1   n        300 non-null    int32  
 2   N        300 non-null    int32  
dtypes: float64(1), int32(2)
memory usage: 4.8 KB


* La tabla `sample_2006` en el paquete `estcomp` contiene para cada casilla:
    + el estrato al que pertenece: `stratum`
    + el número de votos que recibió cada partido/coalición: `pan`, `pri_pvem`, 
    `panal`, `prd_pt_convergencia`, `psd` y la columna `otros` indica el 
    número de votos nulos o por candidatos no registrados.
    + el total de votos registrado en la casilla: `total`.

In [5]:
#Se obtiene dataset original
file_path = "datos\\sample_2006.rda"
sample_2006 = pyreadr.read_r(os.path.join(path_asignatura, file_path))
sample_2006 = sample_2006['sample_2006']
sample_2006.head()

,polling_id,stratum,edo_id,rural,pri_pvem,pan,panal,prd_pt_conv,psd,otros,total
0,74593,106.0,16,1.0,47,40,0,40,0,9,136
1,109927,194.0,27,0.0,131,10,0,147,1,8,297
2,112039,199.0,28,0.0,51,74,2,57,2,2,188
3,86392,141.0,20,1.0,145,64,2,139,1,14,365
4,101306,176.0,24,0.0,51,160,0,64,14,1,290


In [11]:
sample_2006.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   polling_id   7200 non-null   int32  
 1   stratum      7200 non-null   float64
 2   edo_id       7200 non-null   int32  
 3   rural        7200 non-null   float64
 4   pri_pvem     7200 non-null   int32  
 5   pan          7200 non-null   int32  
 6   panal        7200 non-null   int32  
 7   prd_pt_conv  7200 non-null   int32  
 8   psd          7200 non-null   int32  
 9   otros        7200 non-null   int32  
 10  total        7200 non-null   int32  
dtypes: float64(2), int32(9)
memory usage: 365.8 KB


Una de las metodolgías de estimación que se usa en el conteo rápido es **estimador de razón**, y se contruyen intervalos de 95% de confianza usando el método normal con error estándar bootstrap. En este ejercicio debes construir intervalos usando este procedimiento.
<br>Para cada candidato:
1) Calcula el estimador de razón combinado, para muestreo estratificado la fórmula es:
$$\hat{p}=\frac{\sum_h \frac{N_h}{n_h} \sum_i Y_{hi}}{\sum_h \frac{N_h}{n_h} \sum_i X_{hi}}$$
<br>Donde:
* $\hat{p}$ es la estimación de la proporción de votos que recibió el candidato en la elección.
* $Y_{hi}$ es el número total de votos que recibió el candidato en la $i$-ésima casillas, que pertence al $h$-ésimo estrato.
* $X_{hi}$ es el número total de votos en la $i$-ésima casilla, que pertence al $h$-ésimo estrato.
* $N_h$ es el número total de casillas en el $h$-ésimo estrato.
* $n_h$ es el número de casillas del $h$-ésimo estrato que se seleccionaron en la muestra.

2) Utiliza **bootstrap** para calcular el error estándar, y reporta tu 
estimación del error.
    + Genera 1000 muestras bootstrap.
    + Recuerda que las muestras bootstrap tienen que tomar en cuenta la metodología que se utilizó en la selección de la muestra original, en este caso, lo que implica es que debes tomar una muestra aleatoria independiente dentro de cada estrato.
3) Construye un intervalo del 95% de confianza utilizando el método normal. Revisa si el supuesto de normalidad es razonable.
4) Reporta tus intervalos en una tabla. 

**Paso 1**: Se estiman las proporciones para todos los partidos

In [7]:
#Seleccionando columnas relevantes
partidos = ['pri_pvem', 'pan', 'panal', 'prd_pt_conv', 'psd', 'otros']
prop = []

#Estimación de proporciones
for partido in partidos:
    prop_partido = sample_2006[partido].sum() / sample_2006['total'].sum() * 100
    prop.append(prop_partido)
    
props = pd.DataFrame({'partido':partidos, 'prop':prop})
props.sort_values(by='partido')

,partido,prop
5,otros,2.899497
1,pan,35.917034
2,panal,0.952337
3,prd_pt_conv,35.197530
0,pri_pvem,22.326924
4,psd,2.706678
